In [ ]:
import nltk
nltk.download('stopwords', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)  # NEW: required by newer NLTK
print("Downloaded: stopwords, punkt, punkt_tab")


Downloaded: stopwords, punkt, punkt_tab


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, textwrap
func_dir = "/content/drive/MyDrive/Functions"
os.makedirs(func_dir, exist_ok=True)

nlp_code = textwrap.dedent("""
import re, nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize

# Ensure required data is present (works on fresh runtimes)
def _ensure(token):
    try:
        nltk.data.find(token)
    except LookupError:
        pkg = token.split('/')[-2]  # e.g., 'punkt' or 'punkt_tab'
        nltk.download(pkg, quiet=True)

# stopwords
try:
    _stop = set(stopwords.words('english'))
except LookupError:
    nltk.download('stopwords', quiet=True)
    _stop = set(stopwords.words('english'))

# tokenizers (punkt + punkt_tab for newer NLTK)
_ensure('tokenizers/punkt')
_ensure('tokenizers/punkt_tab')

_stemmer = SnowballStemmer('english')
_alnum = re.compile(r"^[a-zA-Z]+$")  # keep alphabetic tokens only

def preprocessing(text_or_tokens):
    \"""
    Mirrors Lab 2.4 preprocessing:
    - input: raw string or list of tokens
    - lowercase
    - tokenize if string (Punkt)
    - filter: stopwords, non-alpha, tokens <2 chars
    - Snowball stem
    Returns: list[str] of stems
    \"""
    if isinstance(text_or_tokens, str):
        tokens = word_tokenize(text_or_tokens.lower())
    else:
        tokens = [t.lower() for t in text_or_tokens]

    cleaned = []
    for t in tokens:
        if len(t) < 2:
            continue
        if t in _stop:
            continue
        if not _alnum.match(t):
            continue
        cleaned.append(_stemmer.stem(t))
    return cleaned
""")

with open(f"{func_dir}/nlp_functions.py", "w") as f:
    f.write(nlp_code)

print("Saved -> /content/drive/MyDrive/Functions/nlp_functions.py")


Mounted at /content/drive
Saved -> /content/drive/MyDrive/Functions/nlp_functions.py


In [ ]:
import sys, importlib
if "/content/drive/MyDrive/Functions" not in sys.path:
    sys.path.append("/content/drive/MyDrive/Functions")

import nlp_functions as fun
importlib.reload(fun)

fun.preprocessing("This is a quick TEST: running, runs, runner! 123")


['quick', 'test', 'run', 'run', 'runner']

In [ ]:
# === Lab 2.x: Imports ===
import pandas as pd
import numpy as np
import re, string, json, math
from collections import Counter

# Text libs
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Our lab helper
import sys, importlib
if "/content/drive/MyDrive/Functions" not in sys.path:
    sys.path.append("/content/drive/MyDrive/Functions")
import nlp_functions as fun
importlib.reload(fun)


<module 'nlp_functions' from '/content/drive/MyDrive/Functions/nlp_functions.py'>

In [ ]:
# === Re-read with explicit column names (file has no header) ===
import pandas as pd, os

raw_path = "/content/Restaurant_reviews_R.csv"
assert os.path.exists(raw_path), f"File not found: {raw_path}"

colnames = [
    "Stars",           # e.g., 5.0, 3.0
    "ReviewDate",      # e.g., 2008-09-29 09:33:04
    "Review",          # the full review text
    "BusinessName",    # e.g., Pizza Hut
    "City",            # e.g., Orlando
    "State",           # e.g., FL
    "Zip",             # e.g., 32819
    "Price"            # e.g., 3.50  (keep as string for now)
]

df = pd.read_csv(
    raw_path,
    engine="python",
    sep=",",
    header=None,           # <-- important: there is no header row
    names=colnames,        # use our clean names
    quotechar='"',
    escapechar="\\",
    on_bad_lines="skip",
    encoding="utf-8",
    dtype=str
)

print("Loaded shape:", df.shape)
df.head(5)


Loaded shape: (63230, 8)


,Stars,ReviewDate,Review,BusinessName,City,State,Zip,Price
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.50
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.00
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.50
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.00
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.50


In [ ]:
# === Data overview ===
# Objective: confirm structure, detect obvious issues (nulls, types).
df.info()
df.isna().sum().to_frame("null_count").T
df.head(5)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63230 entries, 0 to 63229
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Stars         63230 non-null  object
 1   ReviewDate    63230 non-null  object
 2   Review        63229 non-null  object
 3   BusinessName  63229 non-null  object
 4   City          63227 non-null  object
 5   State         63227 non-null  object
 6   Zip           63227 non-null  object
 7   Price         63226 non-null  object
dtypes: object(8)
memory usage: 3.9+ MB


,Stars,ReviewDate,Review,BusinessName,City,State,Zip,Price
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.50
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.00
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.50
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.00
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.50


In [ ]:
# === Text normalization (pre-tokenization) ===
# Rationale: Labs perform contraction expansion before tokenization; whitespace normalized for consistency.

import re

_CONTRACTIONS = {
    "can't": "can not", "won't": "will not", "n't": " not", "'re": " are",
    "'s": " is", "'d": " would", "'ll": " will", "'t": " not", "'ve": " have", "'m": " am"
}

def expand_contractions(text: str) -> str:
    if not isinstance(text, str):
        return ""
    t = text
    for k, v in _CONTRACTIONS.items():
        t = re.sub(k, v, t, flags=re.IGNORECASE)
    return t

def normalize_ws(text: str) -> str:
    return re.sub(r"\s+", " ", str(text)).strip()

review_col = "Review"   # fixed after re-read with named columns
df["_review_raw"] = df[review_col].fillna("").map(normalize_ws).map(expand_contractions)

df[[review_col, "_review_raw"]].head(5)


,Review,_review_raw
0,its the best pizza hut i'v found in florida al...,its the best pizza hut i'v found in florida al...
1,"This place is an interesting combo. The chef, ...","This place is an interesting combo. The chef, ..."
2,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut is great! You get huge pizzas for yo...
3,Always a great place to taste some tea with fr...,Always a great place to taste some tea with fr...
4,Delish! Stopped in on a random Sunday and so g...,Delish! Stopped in on a random Sunday and so g...


In [ ]:
# === Stemming pipeline (Lab 2.4 helper) ===
# Output: list of stems per review.

df["_stems_list"] = df["_review_raw"].apply(fun.preprocessing)

# Global stem frequency for the corpus
from collections import Counter
all_stems = [t for row in df["_stems_list"] for t in row]
stem_counts = Counter(all_stems)

len(all_stems), len(stem_counts)


(3182402, 34189)

In [ ]:
# === Feature selection: top-K stems by corpus frequency ===
# Rationale: rubric requires keeping only the top 500 stems overall.

TOP_K = 500
top_stems = set([w for w, _ in stem_counts.most_common(TOP_K)])

def keep_top_stems(tokens):
    return [t for t in tokens if t in top_stems]

df["_stems_top"] = df["_stems_list"].apply(keep_top_stems)
df["Cleaned_Review_Stems"] = df["_stems_top"].apply(lambda toks: " ".join(toks))

# Deliverable preview: first 5 cleaned reviews
df[["Cleaned_Review_Stems"]].head(5)


,Cleaned_Review_Stems
0,best pizza found alway got great deal offer st...
1,place chef incred cook food perfect tast break...
2,pizza great get huge pizza money servic quick ...
3,alway great place tast tea friend staff friend
4,stop sunday glad eat found make good choic eat...


In [ ]:
# confirm top-K
TOP_K, len(top_stems)


(500, 500)

In [ ]:
# === Persist intermediate result ===
# Output file: cleaned_restaurant_review_stems.csv
# Includes original columns + Cleaned_Review_Stems.

stems_out = df.copy()
stems_out.to_csv("/content/cleaned_restaurant_review_stems.csv", index=False)
print("Saved -> /content/cleaned_restaurant_review_stems.csv")
stems_out.head(3)


Saved -> /content/cleaned_restaurant_review_stems.csv


,Stars,ReviewDate,Review,BusinessName,City,State,Zip,Price,_review_raw,_stems_list,_stems_top,Cleaned_Review_Stems
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.50,its the best pizza hut i'v found in florida al...,"[best, pizza, hut, found, florida, alway, got,...","[best, pizza, found, alway, got, great, deal, ...",best pizza found alway got great deal offer st...
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.00,"This place is an interesting combo. The chef, ...","[place, interest, combo, chef, david, incred, ...","[place, chef, incred, cook, food, perfect, tas...",place chef incred cook food perfect tast break...
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.50,Pizza Hut is great! You get huge pizzas for yo...,"[pizza, hut, great, get, huge, pizza, money, s...","[pizza, great, get, huge, pizza, money, servic...",pizza great get huge pizza money servic quick ...


In [ ]:
# === Restaurant-level aggregation ===
# Goal: combine all reviews for the same restaurant into a single document.
# Grouping keys reflect typical schema of this dataset.

group_cols = ["BusinessName", "City", "State"]

def join_text(series):
    return " ".join(series.dropna().astype(str))

df_rest = (
    df.assign(All_Reviews_Text=df["_review_raw"])
      .groupby(group_cols, as_index=False)
      .agg({"All_Reviews_Text": join_text})
)

df_rest.head(5), df_rest.shape


(             BusinessName       City State  \
 0                1/2 Roll  Vancouver    BC   
 1              21 Nickels  Watertown    MA   
 2  24 Hour Pizza Delivery     Boston    MA   
 3   3 Monkeys Pub & Grill  Vancouver    WA   
 4     5th St Ribs-N-Blues    Atlanta    GA   
 
                                     All_Reviews_Text  
 0  The ambience is kind of funky & interesting. F...  
 1  I like 21 Nickels a ton and they would attract...  
 2  This place needs to be shut down. I could not ...  
 3  Well, it is re-opened. Stopped in today just t...  
 4  First visit here for my weekly southern BBQ fi...  ,
 (1058, 4))

In [ ]:
# === NER feature extraction (optimized) ===
# - Use only the 'ner' component (disable tagger/parser/lemmatizer)
# - Process texts in batches with nlp.pipe and multi-process
# - Cap very long concatenations to avoid huge docs on CPU

import spacy
from collections import Counter

# Load lightweight pipeline with only NER
nlp = spacy.load("en_core_web_sm", disable=["tagger","parser","attribute_ruler","lemmatizer","tok2vec"])
nlp.enable_pipe("ner")
nlp.max_length = 2_000_000  # safety cap; we also hard-cap per text below

NER_LABELS = [
    "PERSON","NORP","FAC","ORG","GPE","LOC","PRODUCT","EVENT","WORK_OF_ART","LAW",
    "LANGUAGE","DATE","TIME","PERCENT","MONEY","QUANTITY","ORDINAL","CARDINAL"
]

# Prepare texts with a hard cap to avoid pathological long docs
TEXT_CAP = 50_000  # characters per restaurant (adjust if needed)
texts = [ (i, (t if isinstance(t, str) else "")[:TEXT_CAP]) for i, t in enumerate(df_rest["All_Reviews_Text"].tolist()) ]

# Run NER in batches
counts = [None] * len(texts)
for (i, doc) in zip([i for i, _ in texts], nlp.pipe((t for _, t in texts), batch_size=64, n_process=2)):
    c = Counter(ent.label_ for ent in doc.ents)
    counts[i] = {lbl: int(c.get(lbl, 0)) for lbl in NER_LABELS}

# Attach columns
for lbl in NER_LABELS:
    df_rest[f"NER_{lbl}"] = [cnt[lbl] for cnt in counts]

# Preview: first 5 restaurants with NER features
ner_cols = [c for c in df_rest.columns if c.startswith("NER_")]
df_rest[group_cols + ner_cols].head(5)



,BusinessName,City,State,NER_PERSON,NER_NORP,NER_FAC,NER_ORG,NER_GPE,NER_LOC,NER_PRODUCT,...,NER_WORK_OF_ART,NER_LAW,NER_LANGUAGE,NER_DATE,NER_TIME,NER_PERCENT,NER_MONEY,NER_QUANTITY,NER_ORDINAL,NER_CARDINAL
0,1/2 Roll,Vancouver,BC,31,11,6,27,8,1,2,...,0,0,2,13,13,2,41,0,7,62
1,21 Nickels,Watertown,MA,30,8,2,48,26,1,2,...,2,0,0,37,24,1,12,1,10,73
2,24 Hour Pizza Delivery,Boston,MA,10,2,1,15,4,0,0,...,0,0,1,4,29,2,31,6,4,31
3,3 Monkeys Pub & Grill,Vancouver,WA,36,1,3,23,12,2,4,...,1,0,0,39,24,1,5,1,3,37
4,5th St Ribs-N-Blues,Atlanta,GA,14,1,5,46,7,2,1,...,2,1,0,7,9,0,5,2,7,29


In [ ]:
# === Persist NER feature table ===
ner_out_path = "/content/restaurant_ner_features.csv"
df_rest.to_csv(ner_out_path, index=False)
print("Saved ->", ner_out_path)

# Quick check
df_rest[group_cols + ner_cols].head(3)


Saved -> /content/restaurant_ner_features.csv


,BusinessName,City,State,NER_PERSON,NER_NORP,NER_FAC,NER_ORG,NER_GPE,NER_LOC,NER_PRODUCT,...,NER_WORK_OF_ART,NER_LAW,NER_LANGUAGE,NER_DATE,NER_TIME,NER_PERCENT,NER_MONEY,NER_QUANTITY,NER_ORDINAL,NER_CARDINAL
0,1/2 Roll,Vancouver,BC,31,11,6,27,8,1,2,...,0,0,2,13,13,2,41,0,7,62
1,21 Nickels,Watertown,MA,30,8,2,48,26,1,2,...,2,0,0,37,24,1,12,1,10,73
2,24 Hour Pizza Delivery,Boston,MA,10,2,1,15,4,0,0,...,0,0,1,4,29,2,31,6,4,31


In [ ]:
# === Q7  Cleaned review (lemmas) — sample-based ===
# Goal: generate lemmas for a small sample
import spacy, re
nlp = spacy.load("en_core_web_sm", disable=["tagger","parser","attribute_ruler","tok2vec"])
nlp.enable_pipe("lemmatizer")
alpha = re.compile(r"^[A-Za-z]+$")

# pick a small sample of non-empty reviews
mask = df["_review_raw"].astype(str).str.strip().str.len().gt(0)
sample_idx = df[mask].index[:100]   # <= 100 rows for speed
sample_texts = df.loc[sample_idx, "_review_raw"].str.lower().tolist()

def lemmas_from_doc(doc):
    return " ".join([t.lemma_ for t in doc if len(t.lemma_)>=2 and alpha.match(t.lemma_)])

# run batched, fast
lem_list = []
for doc in nlp.pipe(sample_texts, batch_size=64):
    lem_list.append(lemmas_from_doc(doc))

# write lemmas back only for the sampled rows (others remain blank)
df["Cleaned_Review_Lemmas"] = df.get("Cleaned_Review_Lemmas", "")
df.loc[sample_idx, "Cleaned_Review_Lemmas"] = lem_list

# Show first 5 NON-EMPTY lemma strings (use this as the screenshot)
mask_lem = df["Cleaned_Review_Lemmas"].astype(str).str.strip().str.len().gt(0)
display(df.loc[mask_lem, ["Cleaned_Review_Lemmas"]].head(5))

# Save CSV (okay if only sample rows are populated; assignment asks for the file)
df.to_csv("/content/cleaned_restaurant_review_lemmas.csv", index=False)
print("Saved -> /content/cleaned_restaurant_review_lemmas.csv")



/usr/local/lib/python3.12/dist-packages/spacy/pipeline/lemmatizer.py:188: UserWarning: [W108] The rule-based lemmatizer did not find POS annotation for one or more tokens. Check that your pipeline includes components that assign token.pos, typically 'tagger'+'attribute_ruler' or 'morphologizer'.
  warnings.warn(Warnings.W108)


,Cleaned_Review_Lemmas
0,its the best pizza hut found in florida always...
1,this place is an interesting combo the chef da...
2,pizza hut is great you get huge pizzas for you...
3,always great place to taste some tea with frie...
4,delish stopped in on random sunday and so glad...


Saved -> /content/cleaned_restaurant_review_lemmas.csv


In [ ]:
# Q7 — show first 5 NON-EMPTY lemma strings
mask_lem = df["Cleaned_Review_Lemmas"].astype(str).str.strip().str.len().gt(0)
df.loc[mask_lem, ["Cleaned_Review_Lemmas"]].head(5)


,Cleaned_Review_Lemmas


In [ ]:
df.to_csv("/content/cleaned_restaurant_review_lemmas.csv", index=False)
print("Saved -> /content/cleaned_restaurant_review_lemmas.csv")


In [ ]:
# === Persist lemma-based cleaned reviews ===
lem_out_path = "/content/cleaned_restaurant_review_lemmas.csv"
df.to_csv(lem_out_path, index=False)
print("Saved ->", lem_out_path)
df.head(3)


Saved -> /content/cleaned_restaurant_review_lemmas.csv


,Stars,ReviewDate,Review,BusinessName,City,State,Zip,Price,_review_raw,_stems_list,_stems_top,Cleaned_Review_Stems,_lemmas_list,Cleaned_Review_Lemmas
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.50,its the best pizza hut i'v found in florida al...,"[best, pizza, hut, found, florida, alway, got,...","[best, pizza, found, alway, got, great, deal, ...",best pizza found alway got great deal offer st...,[],
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.00,"This place is an interesting combo. The chef, ...","[place, interest, combo, chef, david, incred, ...","[place, chef, incred, cook, food, perfect, tas...",place chef incred cook food perfect tast break...,[],
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.50,Pizza Hut is great! You get huge pizzas for yo...,"[pizza, hut, great, get, huge, pizza, money, s...","[pizza, great, get, huge, pizza, money, servic...",pizza great get huge pizza money servic quick ...,[],


In [ ]:
# === Token column diagnostics ===
cols_present = [c for c in ["Cleaned_Review_Lemmas","Cleaned_Review_Stems","_review_raw"] if c in df.columns]
print("Columns present:", cols_present)

def nonempty_count(s):
    return s.astype(str).str.strip().str.len().gt(0).sum()

stats = {}
if "Cleaned_Review_Lemmas" in df.columns:
    stats["lemmas_nonempty"] = nonempty_count(df["Cleaned_Review_Lemmas"])
if "Cleaned_Review_Stems" in df.columns:
    stats["stems_nonempty"] = nonempty_count(df["Cleaned_Review_Stems"])
if "_review_raw" in df.columns:
    stats["raw_nonempty"] = nonempty_count(df["_review_raw"])

print(stats)
df[["Cleaned_Review_Lemmas","Cleaned_Review_Stems","_review_raw"]].head(3)




Columns present: ['Cleaned_Review_Lemmas', 'Cleaned_Review_Stems', '_review_raw']
{'lemmas_nonempty': np.int64(0), 'stems_nonempty': np.int64(63158), 'raw_nonempty': np.int64(63229)}


,Cleaned_Review_Lemmas,Cleaned_Review_Stems,_review_raw
0,,best pizza found alway got great deal offer st...,its the best pizza hut i'v found in florida al...
1,,place chef incred cook food perfect tast break...,"This place is an interesting combo. The chef, ..."
2,,pizza great get huge pizza money servic quick ...,Pizza Hut is great! You get huge pizzas for yo...


In [ ]:
# === Frequency-based BOW on stems (rubric-compliant) ===
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

token_col = "Cleaned_Review_Stems"
docs = df[token_col].astype(str).str.strip()
mask = docs.str.len() > 0
docs_nz = docs[mask]

vect = CountVectorizer(
    max_features=1000,         # adjust if your rubric specifies
    lowercase=False,           # already lowercased
    tokenizer=lambda s: s.split(),
    min_df=2                   # ignore one-off tokens
)
X = vect.fit_transform(docs_nz)

# Re-align to full index (zeros for empty rows)
bow_df_nz = pd.DataFrame(
    X.toarray(),
    index=docs_nz.index,
    columns=[f"BOW_{v}" for v in vect.get_feature_names_out()]
)
full_bow = pd.DataFrame(0, index=df.index, columns=bow_df_nz.columns, dtype=int)
full_bow.loc[bow_df_nz.index, :] = bow_df_nz.values

full_bow.shape


/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(63230, 500)

In [ ]:
# === Persist BOW features with identifiers ===
id_candidates = ["BusinessName","City","State","Stars","ReviewDate"]
id_cols = [c for c in id_candidates if c in df.columns]

df_bow = pd.concat(
    [df[id_cols].reset_index(drop=True), full_bow.reset_index(drop=True)],
    axis=1
) if id_cols else full_bow.copy()

out_bow = "/content/review_bow_features.csv"
df_bow.to_csv(out_bow, index=False)
print("Saved ->", out_bow)

# Preview
df_bow.head(5)


Saved -> /content/review_bow_features.csv


,BusinessName,City,State,Stars,ReviewDate,BOW_abl,BOW_absolut,BOW_across,BOW_actual,BOW_ad,...,BOW_worst,BOW_worth,BOW_would,BOW_wrap,BOW_wrong,BOW_year,BOW_yelp,BOW_yes,BOW_yet,BOW_yummi
0,Pizza Hut,Orlando,FL,5.0,2008-09-29 09:33:04,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,La Mongerie Bakery & Bistro,Atlanta,GA,3.0,2013-12-06 23:22:26,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Pizza Hut,Orlando,FL,5.0,2008-11-16 09:44:04,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Teavana,Atlanta,GA,5.0,2016-04-11 17:00:34,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Native Foods Cafe,Happy Valley,OR,4.0,2015-05-26 06:47:56,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
# === Final sanity summary ===
summary = {
    "rows_total": len(df),
    "cleaned_stems_nonempty": int(df["Cleaned_Review_Stems"].str.len().gt(0).sum()),
    "restaurants_total": len(df_rest),
    "ner_feature_cols": len([c for c in df_rest.columns if c.startswith("NER_")]),
    "bow_feature_cols": int(full_bow.shape[1]),
}
summary


{'rows_total': 63230,
 'cleaned_stems_nonempty': 63158,
 'restaurants_total': 1058,
 'ner_feature_cols': 18,
 'bow_feature_cols': 500}

In [ ]:
# === Persist deliverables to Drive ===
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os, shutil

out_dir = "/content/drive/MyDrive/NLP_Assignment2"
os.makedirs(out_dir, exist_ok=True)

paths = {
    "cleaned_restaurant_review_stems.csv": "/content/cleaned_restaurant_review_stems.csv",
    "restaurant_ner_features.csv": "/content/restaurant_ner_features.csv",
    "cleaned_restaurant_review_lemmas.csv": "/content/cleaned_restaurant_review_lemmas.csv",
    "review_bow_features.csv": "/content/review_bow_features.csv",
    "source_restaurant_reviews.csv": "/content/Restaurant_reviews_R.csv",
}

for name, src in paths.items():
    if os.path.exists(src):
        shutil.copy(src, os.path.join(out_dir, name))
        print("Saved ->", os.path.join(out_dir, name))
    else:
        print("Missing (skipped):", name)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved -> /content/drive/MyDrive/NLP_Assignment2/cleaned_restaurant_review_stems.csv
Saved -> /content/drive/MyDrive/NLP_Assignment2/restaurant_ner_features.csv
Saved -> /content/drive/MyDrive/NLP_Assignment2/cleaned_restaurant_review_lemmas.csv
Saved -> /content/drive/MyDrive/NLP_Assignment2/review_bow_features.csv
Saved -> /content/drive/MyDrive/NLP_Assignment2/source_restaurant_reviews.csv


In [ ]:
# === Persist deliverables to Drive ===
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

import os, shutil

out_dir = "/content/drive/MyDrive/NLP_Assignment2"
os.makedirs(out_dir, exist_ok=True)

paths = {
    "cleaned_restaurant_review_stems.csv": "/content/cleaned_restaurant_review_stems.csv",
    "restaurant_ner_features.csv": "/content/restaurant_ner_features.csv",
    "cleaned_restaurant_review_lemmas.csv": "/content/cleaned_restaurant_review_lemmas.csv",
    "review_bow_features.csv": "/content/review_bow_features.csv",
    "source_restaurant_reviews.csv": "/content/Restaurant_reviews_R.csv",
}

for name, src in paths.items():
    if os.path.exists(src):
        shutil.copy(src, os.path.join(out_dir, name))
        print("Saved ->", os.path.join(out_dir, name))
    else:
        print("Missing (skipped):", name)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved -> /content/drive/MyDrive/NLP_Assignment2/cleaned_restaurant_review_stems.csv
Saved -> /content/drive/MyDrive/NLP_Assignment2/restaurant_ner_features.csv
Saved -> /content/drive/MyDrive/NLP_Assignment2/cleaned_restaurant_review_lemmas.csv
Saved -> /content/drive/MyDrive/NLP_Assignment2/review_bow_features.csv
Saved -> /content/drive/MyDrive/NLP_Assignment2/source_restaurant_reviews.csv


In [ ]:
# === screenshots required===

# 1) First 5 cleaned stems (explicitly required)
df[["Cleaned_Review_Stems"]].head(5)

# 2) First 5 restaurants with NER features (explicitly required)
ner_cols = [c for c in df_rest.columns if c.startswith("NER_")]
df_rest[["BusinessName","City","State"] + ner_cols].head(5)

# 3)  First 5 cleaned lemmas
df[["Cleaned_Review_Lemmas"]].head(5)

# 4)  BOW preview
df_bow.head(5)


,BusinessName,City,State,Stars,ReviewDate,BOW_abl,BOW_absolut,BOW_across,BOW_actual,BOW_ad,...,BOW_worst,BOW_worth,BOW_would,BOW_wrap,BOW_wrong,BOW_year,BOW_yelp,BOW_yes,BOW_yet,BOW_yummi
0,Pizza Hut,Orlando,FL,5.0,2008-09-29 09:33:04,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,La Mongerie Bakery & Bistro,Atlanta,GA,3.0,2013-12-06 23:22:26,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Pizza Hut,Orlando,FL,5.0,2008-11-16 09:44:04,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Teavana,Atlanta,GA,5.0,2016-04-11 17:00:34,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Native Foods Cafe,Happy Valley,OR,4.0,2015-05-26 06:47:56,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
print("Shape:", df.shape)
df.head(5)


Shape: (63230, 14)


,Stars,ReviewDate,Review,BusinessName,City,State,Zip,Price,_review_raw,_stems_list,_stems_top,Cleaned_Review_Stems,_lemmas_list,Cleaned_Review_Lemmas
0,5.0,2008-09-29 09:33:04,its the best pizza hut i'v found in florida al...,Pizza Hut,Orlando,FL,32819,3.50,its the best pizza hut i'v found in florida al...,"[best, pizza, hut, found, florida, alway, got,...","[best, pizza, found, alway, got, great, deal, ...",best pizza found alway got great deal offer st...,[],
1,3.0,2013-12-06 23:22:26,"This place is an interesting combo. The chef, ...",La Mongerie Bakery & Bistro,Atlanta,GA,30308,3.00,"This place is an interesting combo. The chef, ...","[place, interest, combo, chef, david, incred, ...","[place, chef, incred, cook, food, perfect, tas...",place chef incred cook food perfect tast break...,[],
2,5.0,2008-11-16 09:44:04,Pizza Hut is great! You get huge pizzas for yo...,Pizza Hut,Orlando,FL,32819,3.50,Pizza Hut is great! You get huge pizzas for yo...,"[pizza, hut, great, get, huge, pizza, money, s...","[pizza, great, get, huge, pizza, money, servic...",pizza great get huge pizza money servic quick ...,[],
3,5.0,2016-04-11 17:00:34,Always a great place to taste some tea with fr...,Teavana,Atlanta,GA,30326,3.00,Always a great place to taste some tea with fr...,"[alway, great, place, tast, tea, friend, staff...","[alway, great, place, tast, tea, friend, staff...",alway great place tast tea friend staff friend,[],
4,4.0,2015-05-26 06:47:56,Delish! Stopped in on a random Sunday and so g...,Native Foods Cafe,Happy Valley,OR,97086,4.50,Delish! Stopped in on a random Sunday and so g...,"[delish, stop, random, sunday, glad, normal, e...","[stop, sunday, glad, eat, found, make, good, c...",stop sunday glad eat found make good choic eat...,[],


In [ ]:
# aggregate to unique restaurants
df_rest[["BusinessName","City","State","All_Reviews_Text"]].head(5)


,BusinessName,City,State,All_Reviews_Text
0,1/2 Roll,Vancouver,BC,The ambience is kind of funky & interesting. F...
1,21 Nickels,Watertown,MA,I like 21 Nickels a ton and they would attract...
2,24 Hour Pizza Delivery,Boston,MA,This place needs to be shut down. I could not ...
3,3 Monkeys Pub & Grill,Vancouver,WA,"Well, it is re-opened. Stopped in today just t..."
4,5th St Ribs-N-Blues,Atlanta,GA,First visit here for my weekly southern BBQ fi...


In [ ]:
# annotate each row with NER tag categories
ner_cols = [c for c in df_rest.columns if c.startswith("NER_")]
df_rest[["BusinessName","City","State"] + ner_cols].head(5)


,BusinessName,City,State,NER_PERSON,NER_NORP,NER_FAC,NER_ORG,NER_GPE,NER_LOC,NER_PRODUCT,...,NER_WORK_OF_ART,NER_LAW,NER_LANGUAGE,NER_DATE,NER_TIME,NER_PERCENT,NER_MONEY,NER_QUANTITY,NER_ORDINAL,NER_CARDINAL
0,1/2 Roll,Vancouver,BC,31,11,6,27,8,1,2,...,0,0,2,13,13,2,41,0,7,62
1,21 Nickels,Watertown,MA,30,8,2,48,26,1,2,...,2,0,0,37,24,1,12,1,10,73
2,24 Hour Pizza Delivery,Boston,MA,10,2,1,15,4,0,0,...,0,0,1,4,29,2,31,6,4,31
3,3 Monkeys Pub & Grill,Vancouver,WA,36,1,3,23,12,2,4,...,1,0,0,39,24,1,5,1,3,37
4,5th St Ribs-N-Blues,Atlanta,GA,14,1,5,46,7,2,1,...,2,1,0,7,9,0,5,2,7,29


In [ ]:
# create cleaned review (lemmas) as a string
df[["Cleaned_Review_Lemmas"]].head(5)



,Cleaned_Review_Lemmas
0,
1,
2,
3,
4,


In [ ]:
#frequency based Bag of Words features
full_bow.shape[1]

500

In [ ]:
df_bow.head(5)


,BusinessName,City,State,Stars,ReviewDate,BOW_abl,BOW_absolut,BOW_across,BOW_actual,BOW_ad,...,BOW_worst,BOW_worth,BOW_would,BOW_wrap,BOW_wrong,BOW_year,BOW_yelp,BOW_yes,BOW_yet,BOW_yummi
0,Pizza Hut,Orlando,FL,5.0,2008-09-29 09:33:04,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,La Mongerie Bakery & Bistro,Atlanta,GA,3.0,2013-12-06 23:22:26,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Pizza Hut,Orlando,FL,5.0,2008-11-16 09:44:04,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Teavana,Atlanta,GA,5.0,2016-04-11 17:00:34,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Native Foods Cafe,Happy Valley,OR,4.0,2015-05-26 06:47:56,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
summary = {
    "rows_total": len(df),
    "cleaned_stems_nonempty": int(df["Cleaned_Review_Stems"].str.len().gt(0).sum()),
    "restaurants_total": len(df_rest),
    "ner_feature_cols": len([c for c in df_rest.columns if c.startswith("NER_")]),
    "bow_feature_cols": int(full_bow.shape[1]),
}
summary


{'rows_total': 63230,
 'cleaned_stems_nonempty': 63158,
 'restaurants_total': 1058,
 'ner_feature_cols': 18,
 'bow_feature_cols': 500}